In [1]:
pip install pytesseract

In [2]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 90.9 MB/s eta 0:00:00


In [3]:
!apt-get install tesseract-ocr-spa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd tesseract-ocr-spa
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,768 kB of archives.
After this operation, 17.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-spa all 1:4.00~git30-7274cfa-1.1 [951 kB]
Fetched 5,768 kB in 1s (4,915 kB/s)
Selecting previously u

In [4]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [5]:
from PIL import Image
import pytesseract
import os
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf

In [6]:


def extract_text_with_tesseract(pdf_path):
    import fitz  # PyMuPDF
    doc = fitz.open(pdf_path)
    text = ""

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()  # Render page as an image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Use Tesseract to extract text from the image
        text += pytesseract.image_to_string(img, lang='spa')  # 'spa' = Spanish

    return text


pdf_path = "/content/Ezcaray - Vozes.pdf"
extracted_text = extract_text_with_tesseract(pdf_path)

# Save the extracted text
with open("extracted_text_tesseract.txt", "w", encoding="utf-8") as f:
    f.write(extracted_text)

print("Text extracted using Tesseract and saved to 'extracted_text_tesseract.txt'")

Text extracted using Tesseract and saved to 'extracted_text_tesseract.txt'


In [11]:
import re

def clean_text(text):
    # Normalize spaces and remove unwanted characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s.,;:!?¿¡()]', '', text)  # Remove special characters
    return text.strip()

# Load the extracted text
with open("extracted_text_tesseract.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Clean the text
cleaned_text = clean_text(text)
print(cleaned_text[:500])  # Print the first 500 characters to verify

SEÑOR ILVSTRISSIMO. Y Cupado en el exércicio de las Mifliones en el Obifpado de Guadala xararecibiwna de V.S:L. 5. len que nté da noticia de como fu Mageltad (que Dios guarde) fe avia fervido de honrarme con la merced de fu Predicador; y como no e opone la predicación de fu Magefe tad ala Apoftolica,tuvé pordemiobli gaciónadmitir el fávor, rindiendo a V.S.Lel agradecimiento. El Rey mi feñor ( que Dios guarde) hizo la gracia ; mas aV.S.K. fele deber que por mas frutos; que didra la tierra! de Pro


In [12]:
from PIL import Image, ImageDraw, ImageFont

def render_text_image(text, font_path, output_image_path, image_size=(512, 512)):
    # Create a blank image
    image = Image.new("L", image_size, color=255)  # White background
    draw = ImageDraw.Draw(image)

    #Renaissance-style font
    font = ImageFont.truetype(font_path, size=40)

    # Draw the text
    draw.text((10, 10), text, font=font, fill=0)  # Black text
    image.save(output_image_path)

# Example usage
font_path = "/content/IMFellEnglish-Regular.ttf"
output_image_path = "rendered_text.png"
render_text_image(cleaned_text, font_path, output_image_path)  # Render first 500 characters

In [13]:
import cv2
import numpy as np

def add_imperfections(image_path, output_image_path):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Ink bleed
    kernel = np.ones((3, 3), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)

    # Smudging (Gaussian blur)
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Faded text (adjust brightness and contrast)
    alpha = 0.8  # Contrast control
    beta = 50    # Brightness control
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    # Save the degraded image
    cv2.imwrite(output_image_path, image)

# Example usage
degraded_image_path = "degraded_text.png"
add_imperfections(output_image_path, degraded_image_path)

In [14]:
def create_dataset(text, font_path, output_dir, num_samples=100):
    os.makedirs(output_dir, exist_ok=True)
    clean_dir = os.path.join(output_dir, "clean")
    degraded_dir = os.path.join(output_dir, "degraded")
    os.makedirs(clean_dir, exist_ok=True)
    os.makedirs(degraded_dir, exist_ok=True)

    # Split text into chunks
    text_chunks = [text[i:i+500] for i in range(0, len(text), 500)]  # 500 characters per chunk

    for i in range(num_samples):
        chunk = text_chunks[i % len(text_chunks)]  # Cycle through the text chunks
        clean_image_path = os.path.join(clean_dir, f"{i}.png")
        degraded_image_path = os.path.join(degraded_dir, f"{i}.png")

        # Render clean text image
        render_text_image(chunk, font_path, clean_image_path)

        # Add imperfections to create degraded image
        add_imperfections(clean_image_path, degraded_image_path)

    print(f"Dataset created at {output_dir}")

create_dataset(text, font_path, "dataset", num_samples=100)

Dataset created at dataset


In [15]:
def build_generator():
    inputs = layers.Input(shape=(256, 256, 1))

    # Encoder
    e1 = layers.Conv2D(64, 4, strides=2, padding="same")(inputs)
    e1 = layers.LeakyReLU(alpha=0.2)(e1)

    e2 = layers.Conv2D(128, 4, strides=2, padding="same")(e1)
    e2 = layers.BatchNormalization()(e2)
    e2 = layers.LeakyReLU(alpha=0.2)(e2)

    e3 = layers.Conv2D(256, 4, strides=2, padding="same")(e2)
    e3 = layers.BatchNormalization()(e3)
    e3 = layers.LeakyReLU(alpha=0.2)(e3)

    # Decoder
    d1 = layers.Conv2DTranspose(128, 4, strides=2, padding="same")(e3)
    d1 = layers.BatchNormalization()(d1)
    d1 = layers.ReLU()(d1)

    d2 = layers.Conv2DTranspose(64, 4, strides=2, padding="same")(d1)
    d2 = layers.BatchNormalization()(d2)
    d2 = layers.ReLU()(d2)

    outputs = layers.Conv2DTranspose(1, 4, strides=2, padding="same", activation="tanh")(d2)

    return Model(inputs, outputs)

generator = build_generator()
generator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 64)        │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 128)         │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 256)         │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 64, 64, 128)         │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 128, 128, 64)        │         131,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 1)         │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,315,713 (5.02 MB)

 Trainable params: 1,314,561 (5.01 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [16]:
def build_discriminator():
    inputs = layers.Input(shape=(256, 256, 1))

    x = layers.Conv2D(64, 4, strides=2, padding="same")(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, 4, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, 4, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    outputs = layers.Conv2D(1, 4, strides=1, padding="same", activation="sigmoid")(x)

    return Model(inputs, outputs)

discriminator = build_discriminator()
discriminator.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 64)        │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 128)         │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 256)         │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 32, 32, 1)           │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 662,465 (2.53 MB)

 Trainable params: 661,697 (2.52 MB)

 Non-trainable params: 768 (3.00 KB)

In [17]:
# Define loss functions and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy()
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Training loop
def train_gan(generator, discriminator, dataset, epochs=100, batch_size=8):
    for epoch in range(epochs):
        for clean_batch, degraded_batch in dataset:
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                # Generate fake images
                fake_images = generator(clean_batch, training=True)

                # Discriminator output for real and fake images
                real_output = discriminator(degraded_batch, training=True)
                fake_output = discriminator(fake_images, training=True)

                # Discriminator loss
                real_loss = cross_entropy(tf.ones_like(real_output), real_output)
                fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
                disc_loss = real_loss + fake_loss

                # Generator loss
                gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

            # Apply gradients
            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)

            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        print(f"Epoch {epoch + 1}, Discriminator Loss: {disc_loss}, Generator Loss: {gen_loss}")

# Load dataset
def load_dataset(clean_dir, degraded_dir, batch_size=8):
    clean_images = [os.path.join(clean_dir, fname) for fname in os.listdir(clean_dir)]
    degraded_images = [os.path.join(degraded_dir, fname) for fname in os.listdir(degraded_dir)]

    def preprocess_image(image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_png(image, channels=1)
        image = tf.image.resize(image, [256, 256])
        image = tf.cast(image, tf.float32) / 127.5 - 1  # Normalize to [-1, 1]
        return image

    clean_dataset = tf.data.Dataset.from_tensor_slices(clean_images).map(preprocess_image).batch(batch_size)
    degraded_dataset = tf.data.Dataset.from_tensor_slices(degraded_images).map(preprocess_image).batch(batch_size)

    return tf.data.Dataset.zip((clean_dataset, degraded_dataset))

# Train the GAN
dataset = load_dataset("dataset/clean", "dataset/degraded")
train_gan(generator, discriminator, dataset, epochs=100)

Epoch 1, Discriminator Loss: 0.7101912498474121, Generator Loss: 1.4519680738449097
Epoch 2, Discriminator Loss: 0.5158618092536926, Generator Loss: 1.7217036485671997
Epoch 3, Discriminator Loss: 1.328108787536621, Generator Loss: 0.8711020946502686
Epoch 4, Discriminator Loss: 1.103243350982666, Generator Loss: 0.9728996753692627
Epoch 5, Discriminator Loss: 0.993140459060669, Generator Loss: 1.0620049238204956
Epoch 6, Discriminator Loss: 0.9718620777130127, Generator Loss: 1.0968456268310547
Epoch 7, Discriminator Loss: 0.7525038719177246, Generator Loss: 1.3407453298568726
Epoch 8, Discriminator Loss: 0.8089567422866821, Generator Loss: 1.3741068840026855
Epoch 9, Discriminator Loss: 0.7476050853729248, Generator Loss: 1.3715591430664062
Epoch 10, Discriminator Loss: 1.0373213291168213, Generator Loss: 1.3606370687484741
Epoch 11, Discriminator Loss: 0.9997538328170776, Generator Loss: 1.1966084241867065
Epoch 12, Discriminator Loss: 0.8503906726837158, Generator Loss: 1.328289985

In [ ]:
def generate_synthetic_text(generator, clean_image_path):
    clean_image = tf.io.read_file(clean_image_path)
    clean_image = tf.image.decode_png(clean_image, channels=1)
    clean_image = tf.image.resize(clean_image, [256, 256])
    clean_image = tf.cast(clean_image, tf.float32) / 127.5 - 1  # Normalize to [-1, 1]
    clean_image = tf.expand_dims(clean_image, axis=0)  # Add batch dimension

    # Generate degraded image
    degraded_image = generator(clean_image, training=False)
    degraded_image = (degraded_image[0].numpy() + 1) * 127.5  # Denormalize
    degraded_image = np.clip(degraded_image, 0, 255).astype(np.uint8)
    degraded_image = np.squeeze(degraded_image)  # Remove extra dimension


    return degraded_image


synthetic_image = generate_synthetic_text(generator, "/content/rendered_text.png")
Image.fromarray(synthetic_image).save("synthetic_text.png")

In [18]:
#saving models
generator.save("generator_model.h5")
discriminator.save("discriminator_model.h5")
print("Generator and Discriminator models saved.")

Generator and Discriminator models saved.


#genrating synthetic texts from genrator


In [19]:
from PIL import Image, ImageDraw, ImageFont

def render_text_image(text, font_path, output_image_path, image_size=(512, 512)):
    # Create a blank image
    image = Image.new("L", image_size, color=255)  # White background
    draw = ImageDraw.Draw(image)

    # Load a Renaissance-style font
    font = ImageFont.truetype(font_path, size=40)

    # Draw the text
    draw.text((10, 10), text, font=font, fill=0)  # Black text
    image.save(output_image_path)

# Example usage
font_path = "/content/IMFellEnglish-Regular.ttf"

# Split the text into 5 chunks (pages)
text_chunks = [cleaned_text[i:i+2000] for i in range(0, len(cleaned_text), 2000)]  # 2000 characters per page
text_chunks = text_chunks[:5]  # Ensure only 5 pages

# Render each chunk as an image
for i, chunk in enumerate(text_chunks):
    output_image_path = f"clean_page_{i+1}.png"
    render_text_image(chunk, font_path, output_image_path)
    print(f"Rendered page {i+1} to {output_image_path}")

Rendered page 1 to clean_page_1.png
Rendered page 2 to clean_page_2.png
Rendered page 3 to clean_page_3.png
Rendered page 4 to clean_page_4.png
Rendered page 5 to clean_page_5.png


In [21]:


def generate_synthetic_text(generator, clean_image_path):
    # Load and preprocess the clean image
    clean_image = tf.io.read_file(clean_image_path)
    clean_image = tf.image.decode_png(clean_image, channels=1)
    clean_image = tf.image.resize(clean_image, [256, 256])
    clean_image = tf.cast(clean_image, tf.float32) / 127.5 - 1  # Normalize to [-1, 1]
    clean_image = tf.expand_dims(clean_image, axis=0)  # Add batch dimension

    # Generate degraded image
    degraded_image = generator(clean_image, training=False)
    degraded_image = (degraded_image[0].numpy() + 1) * 127.5  # Denormalize
    degraded_image = np.clip(degraded_image, 0, 255).astype(np.uint8)
    degraded_image = np.squeeze(degraded_image)  # Remove extra dimension


    return degraded_image

# Generate synthetic data for 5 pages
for i in range(5):
    clean_image_path = f"clean_page_{i+1}.png"
    synthetic_image = generate_synthetic_text(generator, clean_image_path)

    synthetic_image_path = f"synthetic_page_{i+1}.png"
    Image.fromarray(synthetic_image).save(synthetic_image_path)
    print(f"Generated synthetic page {i+1} and saved to {synthetic_image_path}")

Generated synthetic page 1 and saved to synthetic_page_1.png
Generated synthetic page 2 and saved to synthetic_page_2.png
Generated synthetic page 3 and saved to synthetic_page_3.png
Generated synthetic page 4 and saved to synthetic_page_4.png
Generated synthetic page 5 and saved to synthetic_page_5.png


In [23]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=4580514a7fbd8f37d3a7d2641cc1177e4313f5c177fc4153ec062b61ad346c00
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [24]:
from fpdf import FPDF

# Create a PDF
pdf = FPDF()

# Add each synthetic page to the PDF
for i in range(5):
    synthetic_image_path = f"synthetic_page_{i+1}.png"
    pdf.add_page()
    pdf.image(synthetic_image_path, x=10, y=10, w=180)

# Save the PDF
pdf.output("synthetic_renaissance_text.pdf")
print("Saved 5 pages of synthetic data to 'synthetic_renaissance_text.pdf'")

Saved 5 pages of synthetic data to 'synthetic_renaissance_text.pdf'


# genrating synthetic renassiance paragraphs


In [25]:

text = """“SEÑOR ILVSTRISSIMO.

Y Cupado en el exércicio
de las Mifliones en el
Obifpado de Guadala-
xararecibiwna de V.S:L.

5. len que nté da noticia de
como fu Mageltad (que Dios guarde)
fe avia fervido de honrarme con la
merced de fu Predicador; y como no
e opone la predicación de fu Magefe
tad ala Apoftolica,tuvé pordemiobli
gación'admitir el fávor, rindiendo a
V.S.L'el agradecimiento.

El Rey mi feñor ( que Dios guarde)
hizo la gracia ; mas aV.S.K. fele deber
que por mas frutos; que didra la tierra!
de Promiffion,no los lograra Moyfes,
fi Jofue,y Caleb no los fácaflen: Dos
fácaron el fruto, y de ambos neceffito,
para hallar.vn fimil propotcionado a la
grandeza de V.S.I,"""

# Split text into paragraphs
paragraphs = text.strip().split("\n\n")  # Split by double newlines
print(f"Number of paragraphs: {len(paragraphs)}")

Number of paragraphs: 4


In [26]:
from PIL import Image, ImageDraw, ImageFont

def render_text_image(text, font_path, output_image_path, image_size=(512, 512)):
    # Create a blank image
    image = Image.new("L", image_size, color=255)  # White background
    draw = ImageDraw.Draw(image)

    # Load a Renaissance-style font
    font = ImageFont.truetype(font_path, size=40)

    # Draw the text
    draw.text((10, 10), text, font=font, fill=0)  # Black text
    image.save(output_image_path)

font_path = "/content/IMFellEnglish-Regular.ttf"
# Render each paragraph as an image
for i, paragraph in enumerate(paragraphs):
    output_image_path = f"clean_paragraph_{i+1}.png"
    render_text_image(paragraph, font_path, output_image_path)
    print(f"Rendered paragraph {i+1} to {output_image_path}")

Rendered paragraph 1 to clean_paragraph_1.png
Rendered paragraph 2 to clean_paragraph_2.png
Rendered paragraph 3 to clean_paragraph_3.png
Rendered paragraph 4 to clean_paragraph_4.png


In [27]:
import cv2
import numpy as np

def add_imperfections(image_path, output_image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Ink bleed (dilation)
    kernel = np.ones((3, 3), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)

    # Smudging (Gaussian blur)
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Faded text (adjust brightness and contrast)
    alpha = 0.8  # Contrast control
    beta = 50    # Brightness control
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    # Save the degraded image
    cv2.imwrite(output_image_path, image)

# Add imperfections to each paragraph image
for i in range(len(paragraphs)):
    clean_image_path = f"clean_paragraph_{i+1}.png"
    degraded_image_path = f"degraded_paragraph_{i+1}.png"
    add_imperfections(clean_image_path, degraded_image_path)
    print(f"Added imperfections to paragraph {i+1} and saved to {degraded_image_path}")

Added imperfections to paragraph 1 and saved to degraded_paragraph_1.png
Added imperfections to paragraph 2 and saved to degraded_paragraph_2.png
Added imperfections to paragraph 3 and saved to degraded_paragraph_3.png
Added imperfections to paragraph 4 and saved to degraded_paragraph_4.png


In [29]:
import tensorflow as tf
import numpy as np
from PIL import Image

def generate_synthetic_text(generator, clean_image_path):
    # Load and preprocess the clean image
    clean_image = tf.io.read_file(clean_image_path)
    clean_image = tf.image.decode_png(clean_image, channels=1)
    clean_image = tf.image.resize(clean_image, [256, 256])
    clean_image = tf.cast(clean_image, tf.float32) / 127.5 - 1  # Normalize to [-1, 1]
    clean_image = tf.expand_dims(clean_image, axis=0)  # Add batch dimension

    # Generate degraded image
    degraded_image = generator(clean_image, training=False)
    degraded_image = (degraded_image[0].numpy() + 1) * 127.5  # Denormalize
    degraded_image = np.clip(degraded_image, 0, 255).astype(np.uint8)
    degraded_image = np.squeeze(degraded_image)  # Remove extra dimension

    return degraded_image

# Generate synthetic data for each paragraph
for i in range(len(paragraphs)):
    clean_image_path = f"clean_paragraph_{i+1}.png"
    synthetic_image = generate_synthetic_text(generator, clean_image_path)

    # Save the synthetic image
    synthetic_image_path = f"synthetic_paragraph_{i+1}.png"
    Image.fromarray(synthetic_image).save(synthetic_image_path)
    print(f"Generated synthetic paragraph {i+1} and saved to {synthetic_image_path}")

Generated synthetic paragraph 1 and saved to synthetic_paragraph_1.png
Generated synthetic paragraph 2 and saved to synthetic_paragraph_2.png
Generated synthetic paragraph 3 and saved to synthetic_paragraph_3.png
Generated synthetic paragraph 4 and saved to synthetic_paragraph_4.png


In [30]:
from fpdf import FPDF


pdf = FPDF()

# Add each synthetic paragraph to the PDF
for i in range(len(paragraphs)):
    synthetic_image_path = f"synthetic_paragraph_{i+1}.png"
    pdf.add_page()
    pdf.image(synthetic_image_path, x=10, y=10, w=180)  # Adjust dimensions as needed

# Save the PDF
pdf.output("synthetic_renaissance_paragraphs.pdf")
print("Saved synthetic paragraphs to 'synthetic_renaissance_paragraphs.pdf'")

Saved synthetic paragraphs to 'synthetic_renaissance_paragraphs.pdf'


In [35]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import cv2

def calculate_ssim_psnr(real_image_path, generated_image_path):
    # Load images
    real_image = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    generated_image = cv2.imread(generated_image_path, cv2.IMREAD_GRAYSCALE)

    #  both images should have the same dimensions
    if real_image.shape != generated_image.shape:
        generated_image = cv2.resize(generated_image, (real_image.shape[1], real_image.shape[0]))

    # Calculate SSIM
    ssim_value = ssim(real_image, generated_image, data_range=255)

    # Calculate PSNR
    psnr_value = psnr(real_image, generated_image, data_range=255)

    return ssim_value, psnr_value

# Example usage
real_image_path = "/content/degraded_paragraph_1.png"
generated_image_path = "/content/synthetic_paragraph_1.png"
ssim_value, psnr_value = calculate_ssim_psnr(real_image_path, generated_image_path)
print(f"SSIM: {ssim_value}, PSNR: {psnr_value}")

SSIM: 0.9922099064594826, PSNR: 38.7378163889172
